In [2]:
import pandas as pd
import numpy as np

In [ ]:
ds_path = 'train_admet_all'


df = pd.read_csv(f'../data/{ds_path}.csv')
train_ix = np.load('../data/train_split_idx.npy')
df.loc[:, 'split'] = ['train' if x else 'val' for x in df.index.isin(train_ix)]